In [ ]:
# Desenvolvido por: Marcelo Augusto de Barros Araújo.
# Disciplina: Cálculo Numérico.
# Professor: Marcos Maia.
# Instituição: UABJ.
# Curso: Engenharia da Computação.
# Data: 21/04/25.

In [ ]:
import numpy as np # importando biblioteca

In [ ]:
# --- Definição dos Dados Iniciais ---
# Array de temperaturas conhecidas (pontos x_i)
temperaturas = np.array([30, 35, 40, 45])
# Array de calores específicos correspondentes às temperaturas (pontos y_i)
calores_especificos = np.array([0.99826, 0.99818, 0.99828, 0.99849]) # [cite: 2]

# --- Funções para Interpolação Polinomial de Lagrange ---

def L(k, x, x_pontos):
    """
    Calcula o k-ésimo polinômio base de Lagrange L_k(x).

    Args:
        k (int): Índice do ponto para o qual o polinômio base está sendo calculado.
        x (float): Ponto no qual o polinômio base L_k(x) é avaliado.
        x_pontos (np.array): Array contendo as abscissas (valores de x) dos pontos conhecidos.

    Returns:
        float: O valor de L_k(x) no ponto x.
    """
    termo_total = 1.0
    for i in range(len(x_pontos)):
        if i != k: # O somatório exclui o termo onde i == k
            termo_total *= (x - x_pontos[i]) / (x_pontos[k] - x_pontos[i])
    return termo_total

def polinomio_interpolador(x, x_pontos, y_pontos):
    """
    Calcula o valor do polinômio interpolador de Lagrange P(x) em um ponto x.
    P(x) = Σ (y_k * L_k(x)) para k de 0 a n-1.

    Args:
        x (float): Ponto no qual o polinômio interpolador é avaliado.
        x_pontos (np.array): Array contendo as abscissas (valores de x) dos pontos conhecidos.
        y_pontos (np.array): Array contendo as ordenadas (valores de y) dos pontos conhecidos.

    Returns:
        float: O valor do polinômio interpolador P(x) no ponto x.
    """
    valor_p = 0.0
    for k_idx in range(len(x_pontos)): # Itera sobre todos os pontos conhecidos
        valor_p += y_pontos[k_idx] * L(k_idx, x, x_pontos)
    return valor_p

# --- Parte 1: Cálculo do Calor Específico e Avaliação do Polinômio ---

# Temperatura de interesse para calcular o calor específico
temp_interesse1 = 37.5
# Calcula o calor específico a 37.5°C usando o polinômio interpolador
calor_especifico_37_5 = polinomio_interpolador(temp_interesse1, temperaturas, calores_especificos)
print(f"--- Resolução da Parte 1 ---")
print(f"O calor específico da água a {temp_interesse1}°C é: {calor_especifico_37_5:.7f}") # [cite: 3]

# Temperatura de interesse para avaliar o polinômio p(x)
temp_interesse_p32_5 = 32.5
# Calcula o valor do polinômio p(x) em x = 32.5°C
valor_p_32_5 = polinomio_interpolador(temp_interesse_p32_5, temperaturas, calores_especificos)
print(f"O valor do polinômio p(x) em x = {temp_interesse_p32_5}°C é: {valor_p_32_5:.7f}\n") # [cite: 3]

# --- Verificação com NumPy (polyfit e poly1d) ---
# Encontra os coeficientes do polinômio interpolador de grau 3 usando a função polyfit do NumPy
# O grau é len(temperaturas) - 1, que neste caso é 4 - 1 = 3.
coeficientes_p = np.polyfit(temperaturas, calores_especificos, 3)
# Cria um objeto polinomial a partir dos coeficientes encontrados
p_numpy = np.poly1d(coeficientes_p)

print(f"O polinômio interpolador p(x) encontrado (usando np.polyfit para os coeficientes) é:")
print(p_numpy) # Imprime a representação do polinômio
# Verifica os valores calculados anteriormente usando o polinômio do NumPy
print(f"Verificando p(32.5) com o polinômio do numpy: {p_numpy(32.5):.7f}")
print(f"Verificando p(37.5) com o polinômio do numpy: {p_numpy(37.5):.7f}\n")

# --- Parte 2: Encontrando a Temperatura para um Calor Específico Dado (Busca de Raiz) ---

# Valor do calor específico alvo para o qual queremos encontrar a temperatura
calor_alvo = 0.99837 # [cite: 4]

def f_para_raiz(temp, x_pontos, y_pontos, alvo):
    """
    Define a função f(temp) = P(temp) - alvo, cuja raiz corresponde à temperatura
    onde o calor específico (P(temp)) é igual ao valor 'alvo'.

    Args:
        temp (float): Temperatura na qual a função é avaliada.
        x_pontos (np.array): Abscissas dos pontos de interpolação.
        y_pontos (np.array): Ordenadas dos pontos de interpolação.
        alvo (float): Valor do calor específico alvo.

    Returns:
        float: O valor de P(temp) - alvo.
    """
    return polinomio_interpolador(temp, x_pontos, y_pontos) - alvo

def bisseccao(func, a, b, tol, max_iter, x_pts, y_pts, alvo_val): # [cite: 5]
    """
    Implementa o método da bissecção para encontrar a raiz de uma função 'func'.

    Args:
        func (callable): A função para a qual a raiz está sendo procurada.
                         Deve aceitar (temp, x_pts, y_pts, alvo_val) como argumentos.
        a (float): Limite inferior do intervalo inicial.
        b (float): Limite superior do intervalo inicial.
        tol (float): Tolerância para o critério de parada (erro desejado).
        max_iter (int): Número máximo de iterações permitidas.
        x_pts (np.array): Abscissas dos pontos de interpolação (passado para 'func').
        y_pts (np.array): Ordenadas dos pontos de interpolação (passado para 'func').
        alvo_val (float): Valor alvo (passado para 'func').

    Returns:
        float or None: A raiz aproximada encontrada, ou None se o método falhar
                       ou não convergir dentro do número máximo de iterações.
    """
    # Verifica se f(a) e f(b) têm sinais opostos, uma condição para o método da bissecção
    if func(a, x_pts, y_pts, alvo_val) * func(b, x_pts, y_pts, alvo_val) >= 0:
        print("Método da bissecção falhou: f(a) e f(b) devem ter sinais opostos.")
        return None

    iteracao = 0
    # Critério de parada: a largura do intervalo (b-a)/2 deve ser menor que a tolerância
    # ou o número máximo de iterações é atingido.
    while (b - a) / 2.0 > tol and iteracao < max_iter: # [cite: 5]
        c = (a + b) / 2.0 # Ponto médio do intervalo
        if func(c, x_pts, y_pts, alvo_val) == 0.0: # Raiz exata encontrada
            return c
        elif func(c, x_pts, y_pts, alvo_val) * func(a, x_pts, y_pts, alvo_val) < 0:
            # Se f(c) e f(a) têm sinais opostos, a raiz está no intervalo [a, c]
            b = c
        else:
            # Caso contrário, a raiz está no intervalo [c, b]
            a = c
        iteracao += 1

    if iteracao == max_iter:
        print("Método da bissecção atingiu o número máximo de iterações.")

    return (a + b) / 2.0 # Retorna o ponto médio do intervalo final como a raiz aproximada

print(f"--- Resolução da Parte 2 ---")
# Avalia a função f(T) = p(T) - calor_alvo nos pontos da tabela original
print(f"Valores de p(T) - {calor_alvo} nos pontos da tabela:")
for t in temperaturas:
    valor_f = f_para_raiz(t, temperaturas, calores_especificos, calor_alvo)
    print(f"f({t}) = {polinomio_interpolador(t, temperaturas, calores_especificos):.7f} - {calor_alvo} = {valor_f:.7f}")

# Define o intervalo inicial [a, b] para o método da bissecção.
# Com base nos valores de f(T) impressos acima, escolhemos um intervalo onde f(a) e f(b) tenham sinais opostos.
# f(40) = -0.0000900
# f(45) =  0.0001200
# Portanto, o intervalo [40, 45] é adequado.
intervalo_a = 40.0
intervalo_b = 45.0
tolerancia = 1e-6 # Tolerância desejada para a raiz [cite: 5]
max_iteracoes = 100 # Número máximo de iterações para o método da bissecção

# Avalia a função nos limites do intervalo escolhido para confirmar os sinais opostos
val_a = f_para_raiz(intervalo_a, temperaturas, calores_especificos, calor_alvo)
val_b = f_para_raiz(intervalo_b, temperaturas, calores_especificos, calor_alvo)
print(f"f({intervalo_a}) = {val_a:.7f}")
print(f"f({intervalo_b}) = {val_b:.7f}")

# Aplica o método da bissecção se o intervalo for válido
if val_a * val_b < 0:
    temperatura_raiz = bisseccao(f_para_raiz, intervalo_a, intervalo_b, tolerancia, max_iteracoes,
                                 temperaturas, calores_especificos, calor_alvo)
    if temperatura_raiz is not None:
        print(f"\nA temperatura para a qual o calor específico é {calor_alvo} é aproximadamente: {temperatura_raiz:.6f}°C") # [cite: 4]
        # Verifica o valor do polinômio interpolador na temperatura encontrada
        print(f"Verificação: p({temperatura_raiz:.6f}) = {polinomio_interpolador(temperatura_raiz, temperaturas, calores_especificos):.7f}")
else:
    # Mensagem caso o intervalo inicial não seja adequado
    print(f"\nNão foi possível encontrar um intervalo adequado para a bissecção com [a={intervalo_a}, b={intervalo_b}].")
    print("Verificando outros intervalos possíveis ou se há um erro.")


--- Resolução da Parte 1 ---
O calor específico da água a 37.5°C é: 0.9982119
O valor do polinômio p(x) em x = 32.5°C é: 0.9981931

O polinômio interpolador p(x) encontrado (usando np.polyfit para os coeficientes) é:
            3            2
-9.333e-08 x + 1.34e-05 x - 0.0005907 x + 1.006
Verificando p(32.5) com o polinômio do numpy: 0.9981931
Verificando p(37.5) com o polinômio do numpy: 0.9982119

--- Resolução da Parte 2 ---
Valores de p(T) - 0.99837 nos pontos da tabela:
f(30) = 0.9982600 - 0.99837 = -0.0001100
f(35) = 0.9981800 - 0.99837 = -0.0001900
f(40) = 0.9982800 - 0.99837 = -0.0000900
f(45) = 0.9984900 - 0.99837 = 0.0001200
f(40.0) = -0.0000900
f(45.0) = 0.0001200

A temperatura para a qual o calor específico é 0.99837 é aproximadamente: 42.367280°C
Verificação: p(42.367280) = 0.9983700


---
## Explicação da Atividade 4: Interpolação Polinomial
#Aluno: Marcelo Augusto de Barros Araújo.
Primeiramente, o código define os **dados da tabela** fornecida: as **temperaturas** em graus Celsius e os correspondentes valores de **calor específico** são armazenados em arrays NumPy chamados `temperaturas` e `calores_especificos`.

Na **Parte 1**, tem como objetivo encontrar o **calor específico da água a $37.5^{\circ}C$** e calcular o valor de $p(32.5)$, onde $p(x)$ é um **polinômio interpolador de grau três**. Para isso, o código utiliza a **interpolação de Lagrange**. A função `L(k, x, x_pontos)` calcula o $k$-ésimo **polinômio de base de Lagrange**, $L_k(x)$, usando a fórmula $L_k(x) = \prod_{i=0, i \neq k}^{n} \frac{x - x_i}{x_k - x_i}$. Em seguida, a função `polinomio_interpolador(x, x_pontos, y_pontos)` calcula o valor do **polinômio interpolador $p(x)$** em um ponto $x$ através da soma $p(x) = \sum_{k=0}^{n} y_k L_k(x)$. Assim, para obter o **calor específico a $37.5^{\circ}C$**, o programa chama `polinomio_interpolador(37.5, temperaturas, calores_especificos)`. De maneira análoga, $p(32.5)$ é calculado com `polinomio_interpolador(32.5, temperaturas, calores_especificos)`. Para fins de verificação e para visualizar os coeficientes do polinômio, a função `numpy.polyfit(temperaturas, calores_especificos, 3)` é empregada, o que ajuda a determinar os coeficientes $a, b, c, d$ do **polinômio cúbico** $p(x) = ax^3 + bx^2 + cx + d$.

Na **Parte 2**, a tarefa é determinar a **temperatura $T$ na qual o calor específico é $0.99837$**. Isso é resolvido encontrando a raiz da equação $p(x) = 0.99837$, utilizando o **método da bissecção** com um **erro menor que $10^{-6}$**, e aplicando ao polinômio obtido no item anterior. A função `f_para_raiz(temp, x_pontos, y_pontos, alvo)` é definida como $f(T) = p(T) - \text{alvo}$, onde `alvo` é $0.99837$. A função `bisseccao(func, a, b, tol, max_iter, ...)` implementa o **método da bissecção**. Os parâmetros cruciais são a função `func`, o intervalo inicial `[a, b]`, a **tolerância `tol` (definida como $10^{-6}$)**, e `max_iter`. O método funciona dividindo o intervalo ao meio repetidamente, mantendo a raiz sempre dentro do intervalo, até que a precisão desejada seja alcançada. Para aplicar o método da bissecção, é necessário um intervalo inicial `[a, b]` onde $f(a)$ e $f(b)$ tenham sinais opostos. Ao avaliar $f(T) = p(T) - 0.99837$ nos pontos da tabela original (ou seja, $T \in \{30, 35, 40, 45\}$), observa-se que $f(40)$ e $f(45)$ possuem sinais contrários. Portanto, o intervalo **$[40, 45]$** é um bom ponto de partida. A função `bisseccao` é então chamada com este intervalo.

Enfim, o código imprimirá: o **calor específico calculado para $37.5^{\circ}C$**, o valor de **$p(32.5)$**, os coeficientes do **polinômio $p(x)$ de grau três**, e a **temperatura $T$ para a qual $p(T) \approx 0.99837$**, encontrada com a precisão especificada, juntamente com uma verificação do valor de $p(T)$ na raiz encontrada.

---